In [5]:
print(__doc__)

import numpy as np
import scipy as sp
import pandas as pd
from subprocess import check_output
from sklearn.svm import SVR

import matplotlib.pyplot as plt
%matplotlib inline

import csv

Automatically created module for IPython interactive environment


In [6]:

#read the csv into a dataframe
df = pd.read_csv('groupeddf.csv', parse_dates = ['Date'] )
#print the contents of the dataframe

In [9]:

df.describe()

,Open,High,Low,Close,Volume
count,79322.000000,79322.000000,79322.000000,79322.000000,79322.000000
mean,796.486704,796.924750,796.045823,796.485690,4013.469286
std,2564.123531,2565.299345,2562.938258,2564.110301,11137.480021
min,2.388517,2.390670,2.387560,2.390431,1.000000
25%,95.703103,95.760155,95.622385,95.687901,292.703368
50%,263.770259,263.958052,263.569776,263.769140,1024.355239
75%,659.922287,660.186245,659.522354,659.876600,3644.646337
max,45725.799198,45754.869786,45694.244786,45722.007888,728299.224000


In [10]:
#remove null value (preprocessing)
df = df.dropna()   
df.count() 

Code      79322
Date      79322
Open      79322
High      79322
Low       79322
Close     79322
Volume    79322
dtype: int64

In [11]:
#set index for locating companies
df3=df.set_index("Code")
df3.dtypes

Date      datetime64[ns]
Open             float64
High             float64
Low              float64
Close            float64
Volume           float64
dtype: object

In [12]:
uniqueVals

NameError: name 'uniqueVals' is not defined

In [14]:
# creates an array of company names
uniqueVals = df["Code"].unique() 
# appends the results of all companies
grouped_df=pd.DataFrame()  # empty dataframe
for i in uniqueVals:
    # store DataFrame in list
    grouped_df=grouped_df.append((df3.loc[i,:]).groupby(['Code','Date']).mean())
#grouped_df = append_df.reset_index()
uniqueVals


array(['3IINFOTECH', '8KMILES', 'ABAN', 'ABB', 'ABBOTINDIA', 'ABIRLANUVO',
       'ACC', 'ADANIENT', 'ADANIPORTS', 'ADANIPOWER', 'AHMEDFORGE',
       'AIAENG', 'AIL', 'AJANTPHARM', 'AKZOINDIA', 'ALBK', 'ALEMBICLTD',
       'ALLCARGO', 'ALOKTEXT', 'AMARAJABAT', 'AMBUJACEM', 'AMTEKAUTO',
       'AMTEKINDIA', 'ANANTRAJ', 'ANDHRABANK', 'APLLTD', 'APOLLOHOSP',
       'APOLLOTYRE', 'ARCOTECH', 'ARVIND', 'ARVINDREM', 'ASHOKA',
       'ASHOKLEY', 'ASIANPAINT', 'ASTRAL', 'ASTRAMICRO', 'ATFL',
       'ATULAUTO', 'AURIONPRO', 'AUROPHARMA', 'AXISBANK', 'AXISCADES',
       'BAJAJCORP', 'BAJAJELEC', 'BAJAJFINSV', 'BAJAJHIND', 'BAJAJHLDNG',
       'BAJFINANCE', 'BALKRISIND', 'BALMLAWRIE', 'BALRAMCHIN',
       'BANKBARODA', 'BANKINDIA', 'BATAINDIA', 'BAYERCROP', 'BBL', 'BBTC',
       'BEL', 'BEML', 'BERGEPAINT', 'BFUTILITIE', 'BGRENERGY',
       'BHARATFORG', 'BHARTIARTL', 'BHEL', 'BIOCON', 'BLUEDART',
       'BOMDYEING', 'BOSCHLTD', 'BPCL', 'BPL', 'BRITANNIA', 'CADILAHC',
       'CAIRN', 'CANBK', 'CA

In [15]:
#grouped_CHAMBLFERT= (grouped_df.loc["CHAMBLFERT",:]).drop(['Code', index = False])
grouped_infotech = (grouped_df.loc["3IINFOTECH",:]).reset_index()

In [16]:
grouped_infotech.to_csv('infotech.csv', encoding='utf-8', index=False) #converting into csv

In [17]:
#plt.switch_backend('newbackend') 
import csv
dates = []
prices = []

def get_data(filename):
	with open(filename, 'r') as csvfile:
		csvFileReader = csv.reader(csvfile)
		next(csvFileReader)	# skipping column names
		for row in csvFileReader:
			dates.append(int(row[0].split('-')[2]))
			prices.append(float(row[1]))
	return

get_data('infotech.csv')


In [19]:
def create_datasets(dates,prices):
    train_size=int(0.90*len(dates))
    print (train_size)
    TrainX,TrainY=[],[]
    TestX,TestY=[],[]
    cntr=0
    for date in dates:
        if cntr<train_size:
            TrainX.append(date)
        else:
            TestX.append(date)
        cntr = cntr+1
    cntr=0
    for price in prices:
        if cntr<train_size:
            TrainY.append(price)
        else:
            TestY.append(price)
        cntr = cntr+1
        
    return TrainX,TrainY,TestX,TestY

In [21]:
 TrainX,TrainY,TestX,TestY = create_datasets(dates, prices)
 print (TestX)

142
[8, 9, 10, 11, 14, 15, 16, 18, 21, 22, 23, 24, 28, 29, 30, 1]


In [22]:
def predict_price_test(dates, prices, x):
	dates = np.reshape(dates,(len(dates), 1)) # converting to matrix of n X 1

	svr_rbf = SVR(kernel= 'rbf', C= 1e3, gamma= 0.1) # defining the support vector regression models
	svr_rbf.fit(dates, prices) # fitting the data points in the models
	#print("jhjgj ",svr_rbf.predict(-1)[0])
    #plt.plot(dates, svr_rbf.predict(dates), color= 'red', label= 'RBF model')
    
	return svr_rbf.predict(x)[0]

In [23]:
predicted_price = predict_price_test(dates, prices, 29)  
predicted_price

ValueError: Expected 2D array, got scalar array instead:
array=29.0.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [36]:
predicted_prices = list()
for i in range(30):
    predicted_prices.append(predict_price_test(dates, prices, i+1))
    #predicted_prices = predicted_prices.append(val)
   # predicted_prices = predicted_prices.append(predict_price_test(dates, prices, i))
#plt.plot(predicted_prices)

In [37]:
predicted_prices

[6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405]

In [24]:
import math
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
error = math.sqrt(mean_squared_error(prices, predictions))
print('Test MSE: %.3f' % error)

NameError: name 'predictions' is not defined